In [23]:
import redis
import pandas as pd
import re
from tqdm.notebook import tqdm
import threading
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [29]:
rd = redis.Redis(host = "107.22.67.15",port = 6379,password = "cryogen",charset="utf-8", decode_responses=True)
rd.ping()

True

In [30]:
loaded_df = None
try:
    loaded_df = pd.read_csv("target_df.csv",index_col="scopus_id")
except:
    loaded_df = pd.DataFrame({"scopus_id" : pd.Series(dtype = "object"),
              "author_name" : pd.Series(dtype = "object"),
             "title" : pd.Series(dtype = "object"),
             "abstracts" : pd.Series(dtype = "object"),
             "related_field" : pd.Series(dtype = "object"),
             "auth_keywords" : pd.Series(dtype = "object"),
              "publish_day" : pd.Series(dtype = "object"),
              "publish_month" : pd.Series(dtype = "object"),
              "publish_year" : pd.Series(dtype = "object"),
              "ref_count" : pd.Series(dtype = "int64"),
              "ref_authors" : pd.Series(dtype = "object"),
              "ref_titles" : pd.Series(dtype = "object"),
              "cited_count" : pd.Series(dtype = "int64"),
             })
    loaded_df = loaded_df.set_index("scopus_id")
loaded_df

,author_name,title,abstracts,related_field,auth_keywords,publish_day,publish_month,publish_year,ref_count,ref_authors,ref_titles,cited_count
scopus_id,,,,,,,,,,,,


In [52]:
def multiThreadQuery(thread_size = 8,mykey = "scopus:"):
    cache = set([re.sub(mykey,"",k) for k in rd.keys(mykey + "*")])
    pull_keys = np.array(list(cache - set(loaded_df.index.values.astype(str))))
    my_zipper = []
    
    def pull(key):
        nonlocal my_zipper
        result = rd.hgetall(f"{mykey}{key}")
        my_zipper.append((key,result['author_name'],result['title'],result['abstracts'],result['related_field'],result['auth_keywords'],result['publish_day'],result['publish_month'],result['publish_year'],result['ref_count'],result["ref_authors"],result["ref_titles"],result['cited_count']))
        
    with tqdm(total = len(pull_keys)) as pbar:
        with ThreadPoolExecutor(max_workers=thread_size) as executor:
            future = [executor.submit(pull,key) for key in pull_keys]
            for f in as_completed(future):
                pbar.update(1)     
    col = ["scopus_id","author_name","title","abstracts","related_field","auth_keywords","publish_day","publish_month","publish_year","ref_count","ref_authors","ref_titles","cited_count"]
    arr = list(zip(*my_zipper))
    datapoint = {key : list(value) for key, value in zip(col,zip(*my_zipper))}
    added_df = pd.DataFrame(datapoint)
    if not added_df.empty:
        added_df['scopus_id'] = added_df['scopus_id'].astype(str)
        added_df = added_df.set_index('scopus_id')
        saved_df = pd.concat([loaded_df,added_df],axis = 0)
        saved_df.to_csv("target_df.csv")
    
    return 

In [55]:
multiThreadQuery(2048,"scopus:")

  0%|          | 0/19200 [00:00<?, ?it/s]